# Systems Programming

## Lecture 13: More Pointers and Arrays

### Amir Atapour-Abarghouei

amir.atapour-abarghouei@durham.ac.uk


# Recap - Pointers

- Pointers
- Pointer Assignment
- Pointers as Arguments
- Pointer Arithmetic
- Pointers to Pointers
- Dangers of Pointers

**Breaking things is easy!**

# Recap - Pointer Arithmetic

```
int a[10];
int *pa;
```
This pair of statements are equivalent (+1 translates to +4 bytes (1 int)):
```
pa = &a[1];
pa = (a+1);
```
<center><img src="images/point2second.png" alt="point to array" width="600"/></center>

## Today

# More on Pointers and Arrays

# Pointers and `const`

The `const` keyword is used a differently when pointers are involved.

- These two declarations are equivalent:

```c
const int *ptr_a;
int const *ptr_a;
```

- These two, however, are **Not** equivalent:

```c
int const *ptr_a;
int *const ptr_b;
```

# Pointers and `const`

- These two are **Not** equivalent:

```c
int const *ptr_a;
int *const ptr_b;
```

- In the first example, the `int`(i.e. `*ptr_a`) is `const`.
    - We cannot do `*ptr_a = 123`.
- In the second example, the pointer itself is `const`.
    - We can change `*ptr_b` just fine, but you cannot change (using pointer arithmetic, e.g. `ptr_b++`) the pointer itself.

# Recap - Arrays and Strings

```c
char a[] = "Hello worlds";
char b[13];
char *c;
c = a;
```
- This will set pointer `c` to the same address as `a`
    - `a` is essentially a pointer!

- We can use `strcpy(b,a);` 
    - first argument is the destination - need to `#include <string.h>`

# Function Pointers

It's possible to take the address of a function, too.

- Similarly to arrays, **functions decay to pointers when their names are used**.

So if we wanted the address of `strcpy`, we could just use `strcpy` or `&strcpy`. 

In [4]:
#include<stdio.h>
#include<string.h>

int main(){
    char src[] = "This is a string.", dst[18];
    strcpy(dst, src); //strcpy is basically a pointer
    printf("%s", dst);
    return 0;
}

This is a string.

# Function Pointers

There's syntax for declaring variables whose type is a function pointer.

- This is an ordinary function declaration:
```c
char *strcpy(char *dst, const char *src);
```

- We can now have a function pointer:

```c
char *(*strcpy_ptr)(char *dst, const char *src);

strcpy_ptr =  strcpy;
strcpy_ptr = &strcpy;    // This works too
strcpy_ptr = &strcpy[0]; // But not this, for obvious reasons
```

In [12]:
#include<stdio.h>
#include<string.h>

char *(*strcpy_ptr)(char *dst, const char *src);

int main(){
    char src[] = "This is a string.", dst[18];
    
    strcpy_ptr = strcpy;
    //strcpy_ptr = &strcpy;
    //strcpy_ptr = &strcpy[0];
    strcpy_ptr(dst, src);
    printf("%s", dst);
    return 0;
}

This is a string.

# Function Pointers

```c
char *(*strcpy_ptr)(char *dst, const char *src);
```

- Note the parentheses around `*strcpy_ptr` in the declaration.

    - These separate the `*` indicating return type (`char *`) from the `*` indicating the pointer variable (`*strcpy_ptr` — pointer to function).
    
- As you might expect, a pointer to a pointer to a function has two asterisks inside of the parentheses:

```c
char *(**strcpy_ptr_ptr)(char *, const char *) = &strcpy_ptr;
```



# Function Pointers

Things can get a bit too complicated:

- A function pointer can even be the return value of a function.

```c
char *(*get_strcpy_ptr(void))(char *dst, const char *src);
```

- This is basically the declaration of a function that returns a function pointer.

Since function pointers can get confusing, many use `typedefs` to abstract them:

```c
typedef char *(*strcpy_funcptr)(char *, const char *);

strcpy_funcptr strcpy_ptr = strcpy;
strcpy_funcptr get_strcpy_ptr(void);
```

# Function Pointers

What is important is, just as we have pointers to variables, we can also have pointers to functions!

In [13]:
#include<stdio.h>
void hello_function(int times);

int main(){
    void (*func_ptr)(int);
    func_ptr=hello_function;
    func_ptr(3);
    return 0;
}

void hello_function(int times){
    for(int i=0;i<times;i++)
        printf("Hello, Function Pointer!\n");
}

Hello, Function Pointer!
Hello, Function Pointer!
Hello, Function Pointer!


# Realistic Example - Using `qsort()`

`stdlib.h` contains an implementation of the quicksort algorithm:

```c
void qsort(void *base, size_t nmemb, size_t size,
           int (*compare)(const void *, const void *))
```

- `void *base` is a pointer to the array.
- `size_t nmemb` is the number of elements in the array.
- `size_t size` is the size of each element.
- `int (*compare)(const void *, const void *)` is a function pointer composed of two arguments and returns 0 when the arguments have the same value, <0 when `arg1` comes before `arg2`, and >0 when `arg1` comes after `arg2`.

In [18]:
#include <stdio.h>
#include <stdlib.h>

int compare (const void *, const void *);

int main() {
    int arr[] = {52, 14, 50, 48, 13};
    int num, width, i;
    num = sizeof(arr)/sizeof(arr[0]);
    width = sizeof(arr[0]);
    
    qsort(arr, num, width, compare);
    // we could have used &compare
    for (i = 0; i < 5; i++)
        printf("%d ", arr[i]);
    printf("\n");
    return 0;
}
int compare (const void *arg1, const void *arg2) {
    return *(int *)arg1 - *(int *)arg2;
}

13 14 48 50 52 


# Recap - Pointers and Arrays

In C, if we write `a[x]`, this works by adding `x` to `a` to find the pointer.

- Hence `a[x]` is the same as `*(a+x)`
    - This seems fine with `a[2]`
    - But what if we write `2[a]`?
        - It will compile and run!

# Arrays

- We can also have multi-dimensional arrays in C, e.g.

```c
int matrix[2][3] = {{1,2,3},{4,5,6}};
```
- Now `matrix[0][1]==2`.

- We can have more than 2-dimensional arrays:

```c
int arr3d[3][2][4] = {
    {{1, 2, 3, 4}, {5, 6, 7, 8}},
    {{9, 10, 11, 12}, {13, 14, 15, 16}},
    {{17, 18, 19, 20}, {21, 22, 23, 24}}
};
```

# Multi-Dimensional Arrays

- We can have more than 2-dimensional arrays:

```c
int arr3d[3][2][4] = {
    {{1, 2, 3, 4}, {5, 6, 7, 8}},
    {{9, 10, 11, 12}, {13, 14, 15, 16}},
    {{17, 18, 19, 20}, {21, 22, 23, 24}}
};
```
The elements of arr3d will be allocated in memory in the order

`arr3d[0][0][0]`, `arr3d[0][0][1]`, `arr3d[0][0][2]`,

`arr3d[0][0][3]`, `arr3d[0][1][0]`,`arr3d[0][1][1]`, etc.

# Multi-Dimensional Arrays

- How would we print the values in a multi-dimensional array?

In [20]:
#include<stdio.h>

int main(){
    int arr3d[3][2][4] = {
        {{1, 2, 3, 4}, {5, 6, 7, 8}},
        {{9, 10, 11, 12}, {13, 14, 15, 16}},
        {{17, 18, 19, 20}, {21, 22, 23, 24}}
    };
    
    for(int i=0;i<=2;i++)
        for(int j=0;j<=1;j++)
            for(int k=0;k<=3;k++){
                printf("the value at arr[%d][%d][%d]: ",i,j,k);
                printf("%d\n",arr3d[i][j][k]);
            }
}

the value at arr[0][0][0]: 1
the value at arr[0][0][1]: 2
the value at arr[0][0][2]: 3
the value at arr[0][0][3]: 4
the value at arr[0][1][0]: 5
the value at arr[0][1][1]: 6
the value at arr[0][1][2]: 7
the value at arr[0][1][3]: 8
the value at arr[1][0][0]: 9
the value at arr[1][0][1]: 10
the value at arr[1][0][2]: 11
the value at arr[1][0][3]: 12
the value at arr[1][1][0]: 13
the value at arr[1][1][1]: 14
the value at arr[1][1][2]: 15
the value at arr[1][1][3]: 16
the value at arr[2][0][0]: 17
the value at arr[2][0][1]: 18
the value at arr[2][0][2]: 19
the value at arr[2][0][3]: 20
the value at arr[2][1][0]: 21
the value at arr[2][1][1]: 22
the value at arr[2][1][2]: 23
the value at arr[2][1][3]: 24


# Multi-Dimensional Arrays

```c
int arr3d[3][2][4] = {
    {{1, 2, 3, 4}, {5, 6, 7, 8}},
    {{9, 10, 11, 12}, {13, 14, 15, 16}},
    {{17, 18, 19, 20}, {21, 22, 23, 24}}
};
```

- `&arr3d[i][j][k]` is the same as `&arr3d[0][0][0]+(i*2*4)+j*4+k`


# Multi-Dimensional Arrays

```c
int arr3d[3][2][4] = {
    {{1, 2, 3, 4}, {5, 6, 7, 8}},
    {{9, 10, 11, 12}, {13, 14, 15, 16}},
    {{17, 18, 19, 20}, {21, 22, 23, 24}}
};
```

How would you create a variable to store these?

- What is the type of `arr3d[0][0][0]`?

- What is the type of `arr3d[0][0]`?

- What is the type of `arr3d[0]`?

- What is the type of `arr3d`?

In [24]:
int main(){
    
    int arr3d[2][3][4] = {
        {{1, 2, 3, 4}, {5, 6, 7, 8}, {9, 10, 11, 12}},
        {{13, 14, 15, 16}, {17, 18, 19, 20}, {21, 22, 23, 24}}
    };

    //What are the types of the following?
    //How would you create a variable to store them?
    int zerod = arr3d[0][0][0];
    int *oned = arr3d[0][0];
    int (*twod)[4] = arr3d[0];
    int (*threed)[3][4] = arr3d;
}

# Pointers and Arrays

For further fun with pointers and arrays, take look at:

https://www.oreilly.com/library/view/understanding-and-using/9781449344535/ch04.html

# Summary

- `const` Pointers

- Function Pointers

- `qsort()`

- Pointers and Arrays

- Multi-Dimensional Arrays